<a href="https://colab.research.google.com/github/brunoribas68/POS_IA_UFPR/blob/main/arq_dados/atividade1_Bruno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Identificador automático de idioma</h1>
<p><b>Problema</b>: Dados um texto de entrada, é possível identificar em qual língua o texto está escrito?</p>
<p>
Entrada: "texto qualquer" <br />
Saída: português ou inglês ou francês ou italiano ou...
</p>
<p>&nbsp;</p>

<h2>O processo de Reconhecimento de Padrões</h2>
<p>O objetivo desse trabalho é demonstrar o processo de "construção de atributos" e como ele é fundamental para o <b>Reconhecimento de Padrões (RP)</b>.</p>

<p>Primeiro um conjunto de "amostras" previamente conhecido (classificado)</p>

In [ ]:
#
# amostras de texto em diferentes línguas
#
english = [
"Hello, how are you?",
"I love to read books.",
"The weather is nice today.",
"Where is the nearest restaurant?",
"What time is it?",
"I enjoy playing soccer.",
"Can you help me with this?",
"I'm going to the movies tonight.",
"This is a beautiful place.",
"I like listening to music.",
"Do you speak English?",
"What is your favorite color?",
"I'm learning to play the guitar.",
"Have a great day!",
"I need to buy some groceries.",
"Let's go for a walk.",
"How was your weekend?",
"I'm excited for the concert.",
"Could you pass me the salt, please?",
"I have a meeting at 2 PM.",
"I'm planning a vacation.",
"She sings beautifully.",
"The cat is sleeping.",
"I want to learn French.",
"I enjoy going to the beach.",
"Where can I find a taxi?",
"I'm sorry for the inconvenience.",
"I'm studying for my exams.",
"I like to cook dinner at home.",
"Do you have any recommendations for restaurants?",
]

spanish = [
"Hola, ¿cómo estás?",
"Me encanta leer libros.",
"El clima está agradable hoy.",
"¿Dónde está el restaurante más cercano?",
"¿Qué hora es?",
"Voy al parque todos los días.",
"¿Puedes ayudarme con esto?",
"Me gustaría ir de vacaciones.",
"Este es mi libro favorito.",
"Me gusta bailar salsa.",
"¿Hablas español?",
"¿Cuál es tu comida favorita?",
"Estoy aprendiendo a tocar el piano.",
"¡Que tengas un buen día!",
"Necesito comprar algunas frutas.",
"Vamos a dar un paseo.",
"¿Cómo estuvo tu fin de semana?",
"Estoy emocionado por el concierto.",
"¿Me pasas la sal, por favor?",
"Tengo una reunión a las 2 PM.",
"Estoy planeando unas vacaciones.",
"Ella canta hermosamente.",
"El perro está jugando.",
"Quiero aprender italiano.",
"Disfruto ir a la playa.",
"¿Dónde puedo encontrar un taxi?",
"Lamento las molestias.",
"Estoy estudiando para mis exámenes.",
"Me gusta cocinar la cena en casa.",
"¿Tienes alguna recomendación de restaurantes?",
]

portuguese = [
"Estou indo para o trabalho agora.",
"Adoro passar tempo com minha família.",
"Preciso comprar leite e pão.",
"Vamos ao cinema no sábado.",
"Gosto de praticar esportes ao ar livre.",
"O trânsito está terrível hoje.",
"A comida estava deliciosa!",
"Você já visitou o Rio de Janeiro?",
"Tenho uma reunião importante amanhã.",
"A festa começa às 20h.",
"Estou cansado depois de um longo dia de trabalho.",
"Vamos fazer um churrasco no final de semana.",
"O livro que estou lendo é muito interessante.",
"Estou aprendendo a cozinhar pratos novos.",
"Preciso fazer exercícios físicos regularmente.",
"Vou viajar para o exterior nas férias.",
"Você gosta de dançar?",
"Hoje é meu aniversário!",
"Gosto de ouvir música clássica.",
"Estou estudando para o vestibular.",
"Meu time de futebol favorito ganhou o jogo.",
"Quero aprender a tocar violão.",
"Vamos fazer uma viagem de carro.",
"O parque fica cheio aos finais de semana.",
"O filme que assisti ontem foi ótimo.",
"Preciso resolver esse problema o mais rápido possível.",
"Adoro explorar novos lugares.",
"Vou visitar meus avós no domingo.",
"Estou ansioso para as férias de verão.",
"Gosto de fazer caminhadas na natureza.",
"O restaurante tem uma vista incrível.",
"Vamos sair para jantar no sábado.",
]

A "amostras" de texto precisa ser "transformada" em <b>padrões</b>

<p>Um padrão é um conjunto de características, geralmente representado por um vetor e um conjunto de padrões no formato de tabela. Onde cada linha é um padrão e as colunas as características e, geralmente, na última coluna a <b>classe</b></p>

In [ ]:
texts = english + spanish + portuguese
labels = ["english"] * len(english) + ["spanish"] * len(spanish) + ["portuguese"] * len(portuguese)


O DataFrame do pandas facilita a visualização.

In [ ]:
import re
from collections import Counter
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

# Baixar recursos do NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Função de extração de características
def extract_features(text, language):
    features = {}
    char_counter = Counter(text)
    total_chars = sum(char_counter.values())

    # Frequência de caracteres
    for char, count in char_counter.items():
        features[f'char_freq_{char}'] = count / total_chars

    # Frequência de palavras
    words = word_tokenize(text.lower())
    word_counter = Counter(words)
    total_words = sum(word_counter.values())
    for word, count in word_counter.items():
        features[f'word_freq_{word}'] = count / total_words

    # Comprimento médio das palavras
    word_lengths = [len(word) for word in words]
    features['avg_word_length'] = sum(word_lengths) / len(word_lengths) if word_lengths else 0

    # Frequência de caracteres especiais
    special_chars = re.findall(r'[\!\?\.,;:\-\(\)\[\]]', text)
    special_char_counter = Counter(special_chars)
    for char, count in special_char_counter.items():
        features[f'special_char_freq_{char}'] = count / total_chars

    # Comprimento do texto
    features['text_length'] = len(text)

    # Comprimento médio das sentenças
    sentences = sent_tokenize(text)
    sentence_lengths = [len(sentence) for sentence in sentences]
    features['avg_sentence_length'] = sum(sentence_lengths) / len(sentence_lengths) if sentence_lengths else 0

    return features


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h1>Construção dos atributos</h1>
<p>Esse é o coração desse trabalho e que deverá ser desenvolvido por vocês. Pensem em como podemos "medir" cadas frase/sentença e extrair características que melhorem o resultado do processo de identificação.<p>
<p>Após a criação de cada novo atributo, execute as etapas seguintes e registre as métricas da matriz de confusão. Principalmente acurácia e a precisão.</p>

In [ ]:
# Extração de características
languages = ["english"] * len(english) + ["spanish"] * len(spanish) + ["portuguese"] * len(portuguese)
feature_list = [extract_features(text, lang) for text, lang in zip(texts, languages)]
df = pd.DataFrame(feature_list).fillna(0)  # Substitui NaN por 0
print(df)

    char_freq_H  char_freq_e  char_freq_l  char_freq_o  char_freq_,  \
0      0.052632     0.105263     0.105263     0.157895     0.052632   
1      0.000000     0.095238     0.047619     0.190476     0.000000   
2      0.000000     0.153846     0.000000     0.038462     0.000000   
3      0.000000     0.187500     0.000000     0.000000     0.000000   
4      0.000000     0.062500     0.000000     0.000000     0.000000   
..          ...          ...          ...          ...          ...   
87     0.000000     0.030303     0.000000     0.121212     0.000000   
88     0.000000     0.052632     0.000000     0.105263     0.000000   
89     0.000000     0.078947     0.000000     0.052632     0.000000   
90     0.000000     0.108108     0.027027     0.000000     0.000000   
91     0.000000     0.000000     0.000000     0.090909     0.000000   

    char_freq_   char_freq_h  char_freq_w  char_freq_a  char_freq_r  ...  \
0      0.157895     0.052632     0.052632     0.052632     0.052632  ..

<h2>Treinando o modelo com SVM</h1>
<p>Separando o conjunto de treinamento do conjunto de testes</p>

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Criação de vetores de características e rótulos
X = df.values
y = labels

# Normalização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Redução de dimensionalidade
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_scaled)

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)



Com os conjuntos separados, podemos "treinar" o modelo usando a SVM.

In [ ]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

treinador = svm.SVC()  #algoritmo escolhido
modelo = treinador.fit(X_train, y_train)

#
# score com os dados de treinamento
acuracia = modelo.score(X_train, y_train)
print("Acurácia nos dados de treinamento: {:.2f}%".format(acuracia * 100))

#
# melhor avaliar com a matriz de confusão
y_pred = modelo.predict(X_train)
cm = confusion_matrix(y_train, y_pred)
print(cm)
print(classification_report(y_train, y_pred))

#
# com dados de teste que não foram usados no treinamento
print('métricas mais confiáveis')
y_pred2 = modelo.predict(X_test)
cm = confusion_matrix(y_test, y_pred2)
print(cm)
print(classification_report(y_test, y_pred2))


Acurácia nos dados de treinamento: 98.63%
[[22  0  0]
 [ 0 26  1]
 [ 0  0 24]]
              precision    recall  f1-score   support

     english       1.00      1.00      1.00        22
  portuguese       1.00      0.96      0.98        27
     spanish       0.96      1.00      0.98        24

    accuracy                           0.99        73
   macro avg       0.99      0.99      0.99        73
weighted avg       0.99      0.99      0.99        73

métricas mais confiáveis
[[7 1 0]
 [0 5 0]
 [1 2 3]]
              precision    recall  f1-score   support

     english       0.88      0.88      0.88         8
  portuguese       0.62      1.00      0.77         5
     spanish       1.00      0.50      0.67         6

    accuracy                           0.79        19
   macro avg       0.83      0.79      0.77        19
weighted avg       0.85      0.79      0.78        19

